# SISTER CWL submission

In [77]:
import json
import os
import xml.etree.ElementTree as ET
import pandas as pd
import requests
import IPython
from itertools import groupby
import datetime as dt 

# Import warnings module and ignore warnings in output below
import warnings
warnings.filterwarnings("ignore")

# Import and initialize MAAP class
from maap.maap import MAAP
maap = MAAP(maap_host="sister-api.imgspec.org")

## Load production list with scene URLs


In [3]:
scene_df = pd.read_csv('./sister_production_2_list.csv', encoding='utf-8-sig')
# Strip unicode character
scene_df.l1_granule = scene_df.l1_granule.map(lambda x: x.replace('\ufeff',''))
scene_df.tail()

,Instrument,Type,Area,Flight Line,Platform,l1_granule,Valid URL,Priority,Notes
118,DESIS,Validation,MOBY,DT0623620108_006-20210820T011554-V0215,4,https://sister-ops-workspace.s3.us-west-2.amaz...,True,LOW,MISSING VAL DATA
119,DESIS,Validation,La Crau RadCalNet,DT0729839672_002-20220606T114600-V0215,4,https://sister-ops-workspace.s3.us-west-2.amaz...,True,HIGH,NaN
120,DESIS,Validation,Railroad Valley RadCalNet,DT0735007812_010-20220618T210853-V0215,4,https://sister-ops-workspace.s3.us-west-2.amaz...,True,HIGH,NaN
121,DESIS,Validation,Gobabeb RadCalNet,DT2019020405_002-20190204T084554-V0215,4,https://sister-ops-workspace.s3.us-west-2.amaz...,True,HIGH,NaN
122,DESIS,Terrestrial,UNDERC,DT0488344520_005-20200818T141910-V0210,4,https://sister-ops-workspace.s3.us-west-2.amaz...,True,LOW,NaN


## Workflow configuration generator

The configuration generator function takes as input the URL to the L1 granule and returns a scene identifier along with a workflow configuration for the scene

In [18]:
def gen_config(l1_granule,crid):
    
    '''Generate CWL workflow configuration
    
    Arguments
    
    l1_granule (str): Input scene URL 
    
    Returns:
    
    identifier(str): Unique scene identification code
    
            SISTER_SENSOR_YYYMMDDTHHMMSS
        ex:
    
            SISTER_PRISMA_20200918T100312
            
    input_config (list): List of workflow PGE configurations

    '''
    landsat = 'None'   
    preprocess_queue = "sister-job_worker-16gb"
    segmentation_size = 50
    
    base_name = os.path.basename(l1_granule)
    
    if base_name.startswith('DESIS'):
        sensor = 'DESIS'
        datetime = base_name[31:46]
        
    elif base_name.startswith('PRS'):
        sensor = 'PRISMA'
        datetime = base_name[16:24] + 'T' + base_name[24:30]
        landsat='https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_%s_landsat.tar.gz' % base_name[16:50]
        
    elif base_name.startswith('ang'):
        sensor = 'AVNG'
        datetime = base_name[3:18].upper()
        
    elif base_name.startswith('f'):
        sensor = 'AVCL'
        ''' AVIRIS classic filenames do not contain acquisition times,to be consistent with other
            sensors and to ensure identifier codes are unique a time string is created using other
            numbers in the filename            
        '''     
                
        datetime = "20%sT%s%s%s" % (base_name[1:7],
                                    base_name[8:10],
                                    base_name[11:13],
                                    base_name[14:16])
        
        preprocess_queue = "sister-job_worker-32gb"
        
        #Run large Sierra scenes with larger segmentation size
        if 'f130612' in base_name:
            segmentation_size = 100

    #Check if input file exists
    inputs_exist = requests.head(l1_granule).status_code != 500
    # Check if landsat reference file exists
    if sensor == 'PRISMA':
        inputs_exist &= requests.head(landsat).status_code != 500
        
    if not inputs_exist:
        print('Input file not found.')
        return False,False
        
    input_config =  [
    {
      "step_key": "l1_preprocess",
      "algorithm_id": "sister-preprocess",
      "version": "2.0.0",
      "queue": preprocess_queue,
      "params": {
        "raw_dataset": l1_granule,
        "landsat_dataset": landsat,
        "crid" : crid,
        "identifier": f"SISTER_{sensor}_L1B_RDN_{datetime}_{crid}",
        "username":"anonymous"
      }
    },
    {
      "step_key": "l2_reflectance",
      "algorithm_id": "2.0.0",
      "queue": "sister-job_worker-32gb",
      "version":"sister-dev",
      "input_filter": {
        "radiance_dataset":  f"*RDN*{crid}",
        "location_dataset": "*RDN*LOC",
        "observation_dataset": "*RDN*OBS",          
      },
      "params": {
        "radiance_dataset": None,
        "location_dataset": None,
        "observation_dataset": None,
        "n_cores": 32,
        "segmentation_size": segmentation_size,
        "crid": crid,
        "identifier": f"SISTER_{sensor}_L2A_RFL_{datetime}_{crid}",
        "username":"anonymous"
      },

    },
    {
      "step_key": "l2_resample",
      "algorithm_id": "sister-resample",
      "version":"2.0.0",
      "queue": "sister-job_worker-32gb",
      "input_filter": {
        "reflectance_dataset": f"*_RFL_*{crid}",
        "uncertainty_dataset": "*_RFL*UNC"

      },
      "params": {
        "reflectance_dataset": None,
        "uncertainty_dataset":None,
        "crid" : crid,
        "identifier": f"SISTER_{sensor}_L2A_RSRFL_{datetime}_{crid}",
        "username":"anonymous"

      }
    },
    {
      "step_key": "l2_reflectance_correction",
      "algorithm_id": "2.0.0",
      "version":"sister-dev",
      "queue": "sister-job_worker-32gb",
      "input_filter": {
        "observation_dataset": "*RDN*OBS",
        "reflectance_dataset": f"*RSRFL*{crid}"
      },
      "params": {
        "observation_dataset": None,
        "reflectance_dataset": None,
        "crid" : crid,
        "identifier": f"SISTER_{sensor}_L2A_CORFL_{datetime}_{crid}",
        "username":"anonymous"

      }
    },
    {
      "step_key": "l2_frcover",
      "algorithm_id": "sister-fractional-cover",
      "version": "1.0.0",
      "queue": "sister-job_worker-16gb",
      "input_filter": {
        "reflectance_dataset": "*CORFL*"
      },
      "params": {
        "reflectance_dataset": None,
        "n_cores": 10,
        "refl_scale": 1,
        "normalization": 'brightness',
        "crid" : crid,
        "identifier": f"SISTER_{sensor}_L2B_FRCOVER_{datetime}_{crid}",
        "username":"anonymous"
      }
    }
  ]

    return sensor,datetime,json.dumps(input_config,indent=4)
    

## Generate single CWL workflow configuration

In [20]:
sensor,datetime,input_config = gen_config('https://sister-ops-workspace.s3.us-west-2.amazonaws.com/desis/raw/DESIS-HSI-L1C-DT0488344520_005-20200818T141910-V0210.zip',
                                    987)
print(input_config)
print(sensor,datetime)

[
    {
        "step_key": "l1_preprocess",
        "algorithm_id": "sister-preprocess",
        "version": "sister-dev",
        "queue": "sister-job_worker-16gb",
        "params": {
            "raw_dataset": "https://sister-ops-workspace.s3.us-west-2.amazonaws.com/desis/raw/DESIS-HSI-L1C-DT0488344520_005-20200818T141910-V0210.zip",
            "landsat_dataset": "None",
            "crid": 987,
            "identifier": "SISTER_DESIS_L1B_RDN_20200818T141910_987",
            "username": "anonymous"
        }
    },
    {
        "step_key": "l2_reflectance",
        "algorithm_id": "sister-isofit",
        "queue": "sister-job_worker-32gb",
        "version": "sister-dev",
        "input_filter": {
            "radiance_dataset": "*RDN*987",
            "location_dataset": "*RDN*LOC",
            "observation_dataset": "*RDN*OBS"
        },
        "params": {
            "radiance_dataset": null,
            "location_dataset": null,
            "observation_dataset": null,
     

## Submit single CWL workflow job

In [93]:
identifier = f"SISTER_{sensor}_CWL_{datetime}_{crid}",

job_response = maap.submitJob(
       algo_id="run_sister_workflow",
       version="dev",
       queue="sister-job_worker-8gb",
       identifier= identifier,
       username="anonymous",
       workflow_config=input_config)

print("Submitted %s CWL worflow job" %  identifier)
print("    Submission status: %s" % job_response.status )


Submitted SISTER_DESIS_CWL_20190204T084554_987 CWL worflow job
    Submission status: success


## Create list of granules to run and set CRID

In [108]:
crid = "982"
granules = scene_df[scene_df.Priority == 'HIGH'].l1_granule
print(f"{len(granules)} files in granule list")

21 files in granule list


## Loop through dataframe and start a CWL workflow for each each input scene 

In [92]:
for l1_granule in granules[:1]:
    scene_name = os.path.basename(l1_granule)       
    sensor,datetime,input_config = gen_config(l1_granule,crid)
    identifier = f"SISTER_{sensor}_CWL_{datetime}_{crid}"
    print(f"Submitting {identifier}")

    if identifier:
        job = maap.submitJob(
            algo_id="run_sister_workflow",
            version="dev",
            queue="sister-job_worker-8gb",
            identifier=identifier,
            username="anonymous",
            workflow_config=input_config)
        print(f"\tSubmission status: {job.status}")
       
        

Submitting SISTER_AVNG_CWL_20220405T185108_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220412T190510_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220412T215642_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220420T181856_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220420T212903_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220429T182823_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220503T210356_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220511T182038_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220511T214730_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220517T184026_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220517T213427_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220529T184338_987
	Submission status: {job.status}
Submitting SISTER_AVNG_CWL_20220529T2220

# Print job statuses

### Get list of jobs

In [113]:
job_list = maap.listJobs('anonymous').text
jobs  = json.loads(job_list)

### Set regex pattern to match tags

In [114]:
crid= 985
pattern = f"^SISTER.*{crid}.*TESTING$"

### Cycle through jobs and extracting matching tags

In [115]:
production = {}

for job in jobs['jobs']:
    job_id = list(job.keys())[0]
    tag = job[job_id]['tags'][0]

    if re.match(pattern,tag):
        job_info = {}
        job_info['id'] = job_id
        job_info['status'] = job[job_id]['status'].replace('job-','')

        if 'time_start' in job[job_id]['job']['job_info'].keys():
            start_time = job[job_id]['job']['job_info']['time_start']
        else:
            start_time = '2000-01-01T00:00:00.0Z'
        job_info['start_time'] =dt.datetime.strptime(start_time,'%Y-%m-%dT%H:%M:%S.%fZ')

        if 'time_end' in job[job_id]['job']['job_info'].keys():
            end_time = job[job_id]['job']['job_info']['time_end']
        else:
            end_time = '2000-01-01T00:00:00.0Z'
        job_info['end_time'] =dt.datetime.strptime(end_time,'%Y-%m-%dT%H:%M:%S.%fZ')

        if 'duration' in job[job_id]['job']['job_info'].keys():
            job_duration = float(job[job_id]['job']['job_info']['duration'])/60/60
        else:
            job_duration = -1

        job_info['duration'] =job_duration

        if tag not in production.keys():
            production[tag] = job_info

        elif job_info['start_time'] >  production[tag]['start_time']:
            production[tag] = job_info


### Group tags by tag datetimes and print statuses

In [116]:
tags = list(production.keys())
tags.sort()

for datetime, group in groupby(tags, lambda x: x[-19:]):
    group = [x for x in group]
    key ='_'.join(group[0].split('_')[:2])
    datetime = group[0].split('_')[4]
    print(f"{key}_*_{datetime}")
    for tag in group:
        print(f"\t {tag} status: {production[tag]['status']}")


SISTER_AVCL_*_20170507T010011
	 SISTER_AVCL_L1B_RDN_20170507T010011_985_TAG_TESTING status: completed
	 SISTER_AVCL_L2A_CORFL_20170507T010011_985_TAG_TESTING status: started
	 SISTER_AVCL_L2A_RFL_20170507T010011_985_TAG_TESTING status: completed
	 SISTER_AVCL_L2A_RSRFL_20170507T010011_985_TAG_TESTING status: completed
SISTER_AVNG_*_20180429t045252
	 SISTER_AVNG_L1B_RDN_20180429t045252_985_TAG_TESTING status: completed
	 SISTER_AVNG_L2A_CORFL_20180429t045252_985_TAG_TESTING status: started
	 SISTER_AVNG_L2A_RFL_20180429t045252_985_TAG_TESTING status: completed
	 SISTER_AVNG_L2A_RSRFL_20180429t045252_985_TAG_TESTING status: completed
SISTER_DESIS_*_20190609T195930
	 SISTER_DESIS_L1B_RDN_20190609T195930_985_TAG_TESTING status: completed
	 SISTER_DESIS_L2A_CORFL_20190609T195930_985_TAG_TESTING status: started
	 SISTER_DESIS_L2A_RFL_20190609T195930_985_TAG_TESTING status: completed
	 SISTER_DESIS_L2A_RSRFL_20190609T195930_985_TAG_TESTING status: completed
SISTER_PRISMA_*_20200911T170127
	 S

In [105]:
production

{'SISTER_PRISMA_L2A_RFL_20200911T170127_985_TAG_TESTING': {'id': 'a8390b87-949a-4430-a09a-7522f1f1ccc0',
  'status': 'completed',
  'start_time': datetime.datetime(2023, 2, 9, 15, 55, 45, 713423),
  'end_time': datetime.datetime(2023, 2, 9, 16, 7, 32, 23006),
  'duration': 0.19619710638888888},
 'SISTER_DESIS_L2A_RFL_20190609T195930_985_TAG_TESTING': {'id': 'b952c3c3-cf25-4528-9f41-8656cb444a68',
  'status': 'completed',
  'start_time': datetime.datetime(2023, 2, 9, 15, 55, 54, 61409),
  'end_time': datetime.datetime(2023, 2, 9, 16, 5, 42, 347699),
  'duration': 0.16341285833333333},
 'SISTER_DESIS_L1B_RDN_20190609T195930_985_TAG_TESTING': {'id': 'e3d597ee-ecb9-4264-96bf-bb34f956497d',
  'status': 'completed',
  'start_time': datetime.datetime(2023, 2, 9, 15, 29, 45, 862789),
  'end_time': datetime.datetime(2023, 2, 9, 15, 33, 38, 890531),
  'duration': 0.06472992833333333},
 'SISTER_AVNG_L1B_RDN_20180429t045252_985_TAG_TESTING': {'id': '6cd47ebe-a557-4943-aa4d-0ba185cf817c',
  'status

In [ ]:
def plot_quicklook(job_id):
    result = maap.getJobResult(job_id)
    granule = result.outputs[0] 
    quicklook = result.outputs[0].replace('.tar.gz','.png')
    
    log_url = result.outputs[0].replace('.tar.gz','.log')
    log_response = requests.get(log_url)
    log = log_response.text
    
    return granule,quicklook,log
